## Scenario: Retail Store Sales Prediction

The store runs discounts during weekdays and weekends. We want to:
- Predict sales quantity using discount (%) and whether it's a weekend.
- Optimize discount to achieve maximum sales within a budget.

In [1]:
from typing import Tuple
import numpy as np


def build_dataset() -> Tuple[np.ndarray, np.ndarray]:
    """
    Create the dataset.

    Columns:
      - intercept term (all ones)
      - discount d (%)
      - weekend w (0 or 1)
    Target:
      - sales quantity q

    Returns:
      X: shape (n_samples, 3)
      y: shape (n_samples, 1)
    """
    # Data table (Day, d, w, q)
    rows = [
        # Day,  d,  w,   q
        (1,     5,  0,  50),
        (2,    10,  0,  70),
        (3,    15,  0,  85),
        (4,     5,  1,  80),
        (5,    10,  1, 110),
        (6,    15,  1, 130),
        (7,    20,  1, 150),
    ]

    # Build X with an intercept column of 1s, plus d and w
    X = np.array([[1.0, r[1], float(r[2])] for r in rows], dtype=float)  # (n, 3)
    y = np.array([[r[3]] for r in rows], dtype=float)                    # (n, 1)
    return X, y


def fit_normal_equations(X: np.ndarray, y: np.ndarray) -> np.ndarray:
    """
    Solve (X^T X) theta = X^T y for theta using exact linear algebra.

    Args:
      X: (n, 3) design matrix with columns [1, d, w]
      y: (n, 1) targets (q)

    Returns:
      theta: (3, 1) vector [c, a, b]^T
    """
    # Compute the normal equations components
    XtX = X.T @ X        # shape (3, 3)
    Xty = X.T @ y        # shape (3, 1)

    # Solve the 3x3 linear system for theta
    theta = np.linalg.solve(XtX, Xty)  # (3, 1)

    return theta


def predict(theta: np.ndarray, d: float, w: int) -> float:
    """
    Predict q = c + a*d + b*w using the solved theta.

    Args:
      theta: (3, 1) [c, a, b]
      d: discount (%)
      w: weekend (0 or 1)

    Returns:
      predicted sales quantity (float)
    """
    c, a, b = theta.flatten()
    return c + a * d + b * w


def optimize_discount(theta: np.ndarray,
                      budget_per_day: float,
                      cost_per_1pct: float,
                      weekend: int) -> Tuple[float, float]:
    """
    Solve the simple linear optimization:

      Maximize q(d) = c + a*d + b*w
      Subject to: 0 <= d <= budget_per_day / cost_per_1pct

    Because q is linear and a > 0 in our data, the max is at the upper bound.

    Args:
      theta: (3, 1) [c, a, b]
      budget_per_day: total budget ($/day)
      cost_per_1pct: cost of 1% discount ($ per day)
      weekend: 0 or 1

    Returns:
      (d_star, q_star): optimal discount and resulting predicted quantity
    """
    c, a, b = theta.flatten()
    # Feasible upper bound for discount
    d_max = budget_per_day / cost_per_1pct
    # If a <= 0, the max would be at 0. With our data a > 0, so use the upper bound.
    d_star = d_max if a > 0 else 0.0
    q_star = c + a * d_star + b * weekend
    return d_star, q_star


def main():
    # 1) Build data and matrices
    X, y = build_dataset()

    # 2) Solve normal equations (pure linear algebra)
    theta = fit_normal_equations(X, y)  # shape (3, 1)
    c, a, b = theta.flatten()

    print("=== Solved parameters (theta) from normal equations ===")
    print(f"c (intercept): {c:.6f}")
    print(f"a (per 1% discount): {a:.6f}")
    print(f"b (weekend boost): {b:.6f}\n")

    # 3) Show the explicit linear rule
    print("Fitted rule: q ≈ c + a*d + b*w")
    print(f"q ≈ {c:.4f} + {a:.4f} * d + {b:.4f} * w\n")

    # 4) Make a couple of predictions
    for d, w in [(12, 0), (12, 1), (15, 0), (15, 1)]:
        q_hat = predict(theta, d=d, w=w)
        print(f"Predict(d={d:>2}, w={w}): q ≈ {q_hat:.2f}")
    print()

    # 5) Simple optimization under a daily budget
    budget_per_day = 75.0   # $
    cost_per_1pct  = 5.0    # $ per 1% discount
    for w in [0, 1]:
        d_star, q_star = optimize_discount(theta, budget_per_day, cost_per_1pct, weekend=w)
        print(f"Optimization (weekend={w}): best d = {d_star:.2f}%  →  q ≈ {q_star:.2f}")

    # 6) (Optional) Verify the normal equations were satisfied numerically
    XtX = X.T @ X
    Xty = X.T @ y
    lhs = XtX @ theta
    max_abs_resid = np.max(np.abs(lhs - Xty))
    print(f"\nNormal equations residual (should be ~0): {max_abs_resid:.6e}")


if __name__ == "__main__":
    main()

=== Solved parameters (theta) from normal equations ===
c (intercept): 25.476190
a (per 1% discount): 4.285714
b (weekend boost): 38.452381

Fitted rule: q ≈ c + a*d + b*w
q ≈ 25.4762 + 4.2857 * d + 38.4524 * w

Predict(d=12, w=0): q ≈ 76.90
Predict(d=12, w=1): q ≈ 115.36
Predict(d=15, w=0): q ≈ 89.76
Predict(d=15, w=1): q ≈ 128.21

Optimization (weekend=0): best d = 15.00%  →  q ≈ 89.76
Optimization (weekend=1): best d = 15.00%  →  q ≈ 128.21

Normal equations residual (should be ~0): 1.136868e-13


In [3]:
pip install numpy

  Using cached numpy-2.3.2-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.3.2-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
d = np.array([5,10,15,5,10,15,20],dtype=float)
w  = np.array([0,0,0,1,1,1,1],dtype=float)
q = np.array([50,70,85,80,110,130,150],dtype=float)

In [9]:
y = q.reshape(-1,1)  # (n,1)


In [11]:
ones = np.ones_like(d)
print(ones)

X = np.column_stack((ones,d,w))  # (n,3)
print(X)

[1. 1. 1. 1. 1. 1. 1.]
[[ 1.  5.  0.]
 [ 1. 10.  0.]
 [ 1. 15.  0.]
 [ 1.  5.  1.]
 [ 1. 10.  1.]
 [ 1. 15.  1.]
 [ 1. 20.  1.]]


In [13]:
XtX= X.T @ X  # (3,3)
Xty = X.T @ y  # (3,1)

theta = np.linalg.inv(XtX) @ Xty  # (3,1)
print(theta)    

[[25.47619048]
 [ 4.28571429]
 [38.45238095]]


In [15]:
c,m1,m2 = theta.flatten()

In [16]:
def predict(d_value, w_value):
    return c+m1*d_value + m2*w_value

In [20]:
predict(0, 0)

np.float64(25.47619047619051)